# Expirence 2

# Introduction

## Differential Equations

\begin{equation}
\frac{di_1(t)}{dt}=D(t)\frac{v_1(t)}{L}-\frac{v_0(t)}{L}
\end{equation}

\begin{equation}
\frac{dv_0(t)}{dt}=D(t)\frac{i_1(t)}{C}-\frac{v_0(t)}{RC}
\end{equation}

### First equation in discreet format

\begin{equation}
\frac{i_1[t+step] - i_1[t]}{step}= D[t]\frac{v_1[t]}{L}-\frac{v_0[t]}{L}
\end{equation}
 
\begin{equation}
i_1[t+step]  = i_1[t] + step\cdot \left(D[t]\frac{v_1[t]}{L}-\frac{v_0[t]}{L}\right)
\end{equation}

### Second equation in discreet format
\begin{equation}
\frac{v_0[t+step] - v_0[t]}{step}=\frac{i_1(t)}{C}-\frac{v_0(t)}{RC}
\end{equation}
\begin{equation}
v_0[t+step] = v_0[t] + step\left(\frac{i_1(t)}{C}-\frac{v_0(t)}{RC}\right)
\end{equation}

## Differential Equations Discreet

\begin{equation}
i_1[t+step]  = i_1[t] + step\cdot \left(D[t]\frac{v_1[t]}{L}-\frac{v_0[t]}{L}\right)
\end{equation}

\begin{equation}
v_0[t+step] = v_0[t] + step\left(\frac{i_1(t)}{C}-\frac{v_0(t)}{RC}\right)
\end{equation} 

# 1a
## Dependencies

In [ ]:
import math
import numpy as np
from scipy.optimize import fsolve

# Analysis and plotting modules
import pandas as pd
import plotly


# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD import configs

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor



pd.options.plotting.backend = "plotly"

## State Variables

In [ ]:
initial_state = {
    'i_1': 0, #A
    'v_o': 0, #V
    'D': 0.5  #abs
}

initial_state

In [ ]:
system_params = {
    'step': [0.000001],              #1μs
    'R': [1],                        #ohms
    'L': [math.pow(10,-3)],          #H
    'C': [800*math.pow(10,-6)],      #F
    'vi': [1],                       #V
    'fs': [5 * math.pow(10,3)],       #5kHz
    'Rsh': [38.17],                  #ohms
    'Rs': [61.3*math.pow(10,-3)],    #ohms
    'beta': [86.14 * math.pow(10,-6)], #V/K
    'n': [1.7536],                   #abs
    'Is': [5.68*math.pow(10,-6)],    #A
    'Iphn': [3.1656],                #A
    'Gn': [math.pow(10,3)],          #W/m2
    'T': [298]                       #°K
}


system_params

## State Update Functions

In [ ]:
def s_duty_cycle_a(params, substep, state_history, previous_state, policy_input):
    t = previous_state['timestep']*params['step']
    freq = params['fs']
    period = 1/freq
    dutycicle = 0.5
    if (t%period) < (period*dutycicle):
        pwm = 1
    else:
        pwm = 0
    return 'D', pwm


def s_i1(params, substep, state_history, previous_state, policy_input):
    L = params['L']
    vi = params['vi']
    step = params['step']
    i1 = previous_state['i_1'] + step*((previous_state['D'] * vi / L) - (previous_state['v_o']/L))
    return 'i_1', i1

def s_vo(params, substep, state_history, previous_state, policy_input):
    C = params['C']
    R = params['R']
    step = params['step']
    vo = previous_state['v_o'] + step*((previous_state['i_1']/C) - (previous_state['v_o']/(R*C)))
    return 'v_o', vo

## Partial State Update Blocks

In [ ]:
partial_state_update_blocks = [
    {
        'policies': {},
        'variables': {
            'D': s_duty_cycle_a,
            'i_1': s_i1,
            'v_o': s_vo
        }
    }
]

## Configuration

In [ ]:
sim_config = config_sim({
    "N": 1,
    "T": range(20000),
    "M": system_params
})

del configs[:] # Clear any prior configs
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)
configs[-1].__dict__

## Execution

In [ ]:
exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()

## Simulation Output Preparation

In [ ]:
simulation_result = pd.DataFrame(raw_result)
simulation_result['time'] = simulation_result['timestep'] * system_params['step'][0]
simulation_result.head()

## Simulation Analysis

In [ ]:
simulation_result.plot(kind='line', x='time', y='v_o')

In [ ]:
simulation_result.plot(kind='line', x='time', y='i_1')

In [ ]:
simulation_result.plot(kind='line', x='time', y='D')

# 7

In [ ]:
system_params['fs'] = [100]

In [ ]:
sim_config = config_sim({
    "N": 1,
    "T": range(20000),
    "M": system_params
})

del configs[:] # Clear any prior configs
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()

In [ ]:
simulation_result = pd.DataFrame(raw_result)
simulation_result['time'] = simulation_result['timestep'] * system_params['step'][0]
simulation_result.plot(kind='line', x='time', y='v_o')

# 1.b

In [ ]:
def s_duty_cycle_b(params, substep, state_history, previous_state, policy_input):
    return 'D', 0.5

In [ ]:
partial_state_update_blocks = [
    {
        'policies': {},
        'variables': {
            'D': s_duty_cycle_b,
            'i_1': s_i1,
            'v_o': s_vo
        }
    }
]

del configs[:] # Clear any prior configs
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)
configs[-1].__dict__

In [ ]:
exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()

In [ ]:
simulation_result = pd.DataFrame(raw_result)
simulation_result['time'] = simulation_result['timestep'] * system_params['step'][0]
simulation_result.head()

In [ ]:
simulation_result.plot(kind='line', x='time', y='v_o')

In [ ]:
simulation_result.plot(kind='line', x='time', y='i_1')

# 2
Com a frequencia os valores tem uma oscilação

# 3

We know, that:

\begin{equation}
\frac{di_1(t)}{dt}=D(t)\frac{v_1(t)}{L}-\frac{v_0(t)}{L}
\end{equation}

\begin{equation}
\frac{dv_0(t)}{dt}=D(t)\frac{i_1(t)}{C}-\frac{v_0(t)}{RC}
\end{equation}


aplicando a transferência nas duas EDOs, temos que:

\begin{equation}
sI(s)=D(s)\frac{v_1}{L}-\frac{v_0(s)}{L}
\end{equation}

ou ainda:

\begin{equation}
I(s)=\frac{D(s)v_1-v_0(s)}{Ls}
\end{equation}

na segunda EDO:

\begin{equation}
sV_o(s)=\frac{I(s)}{C}-\frac{v_0(s)}{RC}
\end{equation}

Substituindo, temos que:

\begin{equation}
sV_o(s) = \frac{v_1D(s)-V_o(s)}{CLs} - \frac{V_o(s)}{RC}
\end{equation}

\begin{equation}
sV_o(s) + \frac{V_o(s)}{CLs} + \frac{V_o(s)}{RC} = \frac{v_1D(s)}{CLs}
\end{equation}

\begin{equation}
V_o(s) (s + \frac{V_o(s)}{CLs} + \frac{V_o(s)}{RC}) = \frac{v_1D(s)}{CLs}
\end{equation}

\begin{equation}
V_o(s) (\frac{R+Ls+LCs^2R}{CLsR}) = \frac{v_1D(s)}{CLs}
\end{equation}

\begin{equation}
V_o(s) (\frac{R+Ls+LCs^2R}{R}) = v_1D(s)
\end{equation}

\begin{equation}
G(s) = \frac{V_o(s)}{D(s)} =\frac{Rv_1}{RLCs^2+sL+R}
\end{equation}


# 4

Podemos rescrever a função acima no formato padrão da função de transferência de segunda ordem:

\begin{equation}
G(s) =\frac{Rv_1}{LCs^2+\frac{sL}{R}+1}
\end{equation}

Dessa forma, temos $\tau=\sqrt{LC}$ e para o fator de amortecimento, temos:
\begin{equation}
2\tau\xi = \frac{L}{R}
\end{equation}
\begin{equation}
\xi = \frac{L}{2R\tau}
\end{equation}
\begin{equation}
\xi = \frac{\sqrt{L}}{2R\sqrt{C}}
\end{equation}

Sendo assim, sabemos que a resposta é considerada criticamente amortecida quando $\xi = 1$. Logo:

\begin{equation}
\xi = \frac{\sqrt{L}}{2R\sqrt{C}}=1
\end{equation}
\begin{equation}
R = \frac{\sqrt{L}}{2\xi\sqrt{C}}
\end{equation}
\begin{equation}
R = \frac{\sqrt{L}}{2\sqrt{C}}
\end{equation}

# 5

In [ ]:
system_params['R'] = [0.4, 0.559, 0.7]
system_params

In [ ]:
sim_config = config_sim({
    "N": 1,
    "T": range(20000),
    "M": system_params
})

partial_state_update_blocks = [
    {
        'policies': {},
        'variables': {
            'D': s_duty_cycle_b,
            'i_1': s_i1,
            'v_o': s_vo
        }
    }
]

del configs[:] # Clear any prior configs
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)
configs[-1].__dict__

In [ ]:
exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()

In [ ]:
simulation_result = pd.DataFrame(raw_result)
simulation_result['time'] = simulation_result['timestep'] * system_params['step'][0]
simulation_result.head()

In [ ]:
simulation_result['R'] = simulation_result['subset']
simulation_result['R'].replace({0: 0.4, 1: 0.559, 2:0.7}, inplace=True)
simulation_result.plot(kind='line', x='time', y='i_1', color='R')

# 9

## a e b


In [ ]:
system_params['kp'] = [0.1]  #1/V
system_params['ki'] = [200]  #1/Vs
system_params['kd'] = [0.02] #s/V
system_params['N'] = [10**5] #1/V
system_params['Vref'] = [0.15]  #V
system_params['R'] = [1]

system_params

In [ ]:
initial_state = {
    'i_1': 3.15660026, #A
    'v_o': 0.15, #V
    'D': 0.15, #abs
    'I': 3.15660026,   #A
    'e': 0,
    'ui': 0.15,
    'ud': 0,
    'vref': 0.15
}

initial_state

In [ ]:
def calculate_e(params, previous_state):
    C = params['C']
    R = params['R']
    step = params['step']
    R = previous_state['v_o'] / previous_state['I']
    vo = previous_state['v_o'] + step*((previous_state['i_1']/C) - (previous_state['I']/C))
    e = previous_state['vref'] - vo
    return e

def s_vref(params, substep, state_history, previous_state, policy_input):
    return 'vref', 0.15

def s_I(params, substep, state_history, previous_state, policy_input):
    Iph = params['Iphn']
    Is = params['Is']
    Vt = params['beta'] * params['T']
    Rs = params['Rs']
    n = params['n']
    Rsh = params['Rsh']
    v_o = previous_state['v_o']
    
    def equation(vars):
        I = vars
        eq = I - Iph + Is*(np.exp((v_o + I*Rs)/(n*Vt)) - 1) + (v_o + (I*Rs))/Rsh
        return eq
    
    return 'I', fsolve(equation, previous_state['I'], xtol=1e-6)[0]

def s_vo(params, substep, state_history, previous_state, policy_input):
    C = params['C']
    R = params['R']
    step = params['step']
    R = previous_state['v_o'] / previous_state['I']
    vo = previous_state['v_o'] + step*((previous_state['i_1']/C) - (previous_state['I']/C))
    return 'v_o', vo

def s_e(params, substep, state_history, previous_state, policy_input):
    e = calculate_e(params, previous_state)
    return 'e', e

def s_control_ui(params, substep, state_history, previous_state, policy_input):
    ui = previous_state['ui']+(params['step']*params['ki']*previous_state['e'])
    
    if ui>1:
        ui=1
    
    elif ui < 0:
        ui=0
    return 'ui', ui

def s_control_ud(params, substep, state_history, previous_state, policy_input):
    e = calculate_e(params, previous_state)
    kd = params['kd']
    N = params['N']
    step = params['step']
    
    return 'ud', (previous_state['ud']*(1-(N*step))) + (kd*N*(e-previous_state['e']))


def s_control_D(params, substep, state_history, previous_state, policy_input):
    kd = params['kd']
    N = params['N']
    step = params['step']
    e = calculate_e(params, previous_state)
    
    up = params['kp'] * e
    
    ui = previous_state['ui']+(params['step']*params['ki']*previous_state['e'])
    
    ud = (previous_state['ud']*(1-(N*step))) + (kd*N*(e-previous_state['e']))
    
    d = up+ui+ud
    if d > 1:
        d = 1
        
    elif d < 0:
        d = 0
    return 'D', d
    

In [ ]:
partial_state_update_blocks = [
    {
        'policies': {},
        'variables': {
            'D': s_control_D,
            'i_1': s_i1,
            'v_o': s_vo,
            'I': s_I,
            'e': s_e,
            'ui': s_control_ui,
            'ud': s_control_ud,
            'vref': s_vref
        }
    }
]

In [ ]:
sim_config = config_sim({
    "N": 1,
    "T": range(300000),
    "M": system_params
})

del configs[:] # Clear any prior configs
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)
configs[-1].__dict__

In [ ]:
exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()

In [ ]:
simulation_result = pd.DataFrame(raw_result)
simulation_result.head()

In [ ]:
simulation_result.plot(kind='line', x='timestep', y='v_o')

## c

In [ ]:
system_params['step'] = [0.000005]
system_params

In [ ]:
def s_vref(params, substep, state_history, previous_state, policy_input):
    steps = (previous_state['timestep']+1)*params['step'] // 0.3
    vref = 0.15 + steps*0.03
    if vref > 0.42:
        vref = 0.42
    return 'vref', vref

In [ ]:
partial_state_update_blocks = [
    {
        'policies': {},
        'variables': {
            'D': s_control_D,
            'i_1': s_i1,
            'v_o': s_vo,
            'I': s_I,
            'e': s_e,
            'ui': s_control_ui,
            'ud': s_control_ud,
            'vref': s_vref
        }
    }
]

In [ ]:
sim_config = config_sim({
    "N": 1,
    "T": range(600000),
    "M": system_params
})

del configs[:] # Clear any prior configs
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)
configs[-1].__dict__

In [ ]:
exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()

In [ ]:
simulation_result = pd.DataFrame(raw_result)
simulation_result['time'] = simulation_result['timestep'] * system_params['step'][0]
simulation_result['R'] = simulation_result['v_o'] / simulation_result['I']
simulation_result['P'] = simulation_result['v_o'] * simulation_result['I']
simulation_result.head()

In [ ]:
simulation_result.plot(kind='line', x='time', y='v_o')

In [ ]:
simulation_result.plot(kind='line', x='time', y='vref')

In [ ]:
simulation_result.plot(kind='line', x='time', y='I')

In [ ]:
simulation_result.plot(kind='line', x='time', y='i_1')

In [ ]:
simulation_result.plot(kind='line', x='time', y='R')

In [ ]:
simulation_result.plot(kind='line', x='time', y='P')

In [ ]:
simulation_result.plot(kind='line', x='time', y='D')